In [64]:
import joblib
import category_encoders
import mlflow
import os
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [65]:
from category_encoders import CatBoostEncoder

In [66]:
EXPERIMENT_NAME = 'baseline_improvement'
RUN_NAME = 'base_model'

In [70]:
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

In [68]:
os.environ["MLFLOW_S3_ENDPOINT_URL"] = 'https://storage.yandexcloud.net'
os.environ["AWS_ACCESS_KEY_ID"] = 'YCAJE8CKXGh8XKUmcmGHpV2CX'
os.environ["AWS_SECRET_ACCESS_KEY"] = 'YCNQmCkW71PeIVypo2YsF-4eIHhiRBZZN94pztrj'


mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

In [6]:
#experiment = mlflow.create_experiment(EXPERIMENT_NAME)

RestException: RESOURCE_ALREADY_EXISTS: Experiment(name=baseline_improvement) already exists. Error: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "experiments_name_key"
DETAIL:  Key (name)=(baseline_improvement) already exists.

[SQL: INSERT INTO experiments (name, artifact_location, lifecycle_stage, creation_time, last_update_time) VALUES (%(name)s, %(artifact_location)s, %(lifecycle_stage)s, %(creation_time)s, %(last_update_time)s) RETURNING experiments.experiment_id]
[parameters: {'name': 'baseline_improvement', 'artifact_location': '', 'lifecycle_stage': 'active', 'creation_time': 1720428670032, 'last_update_time': 1720428670032}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [69]:
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

KeyboardInterrupt: 

In [ ]:
experiment

<Experiment: artifact_location='s3://s3-student-mle-20240523-9a4b29d5bb/11', creation_time=1720427881650, experiment_id='11', last_update_time=1720427881650, lifecycle_stage='active', name='baseline_improvement', tags={}>

In [18]:
experiment_id = experiment.experiment_id

In [10]:
experiment_id

'11'

In [61]:
REGISTRY_MODEL_NAME = 'price_prediction_model'

In [20]:
with mlflow.start_run(experiment_id=experiment_id,run_name=RUN_NAME) as run:
    run_id = run.info.run_id
    model_info = mlflow.sklearn.log_model(sk_model=model,artifact_path='models',registered_model_name=REGISTRY_MODEL_NAME)

Successfully registered model 'price_prediction_model'.
2024/07/08 08:46:37 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: price_prediction_model, version 1
Created version '1' of model 'price_prediction_model'.


Базовая модель - модель RandomForestRegressor (fitted_model.pkl) сохранена MLflow Model Registry под именем 'price_prediction_model'

In [5]:
df = pd.read_csv('/home/mle-user/mle_projects/mle-project-sprint-2-v001/mlflow_server/initial_data.csv')

In [6]:
X = df.drop('price',axis=1)
y = df['price']

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)

In [8]:
model = RandomForestRegressor(n_estimators=100)

In [9]:
model.fit(X_train,y_train)

RandomForestRegressor()

In [46]:
prediction = model.predict(X_test)

In [54]:
signature = mlflow.models.infer_signature(X_test,prediction)

/home/mle-user/mle_projects/mle-project-sprint-2-v001/.venv/lib/python3.10/site-packages/mlflow/models/signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  # Saving model with model signature


In [55]:
pip_requirements = '/home/mle-user/mle_projects/mle-project-sprint-2-v001/requirements.txt'

In [56]:
RUN_NAME = 'base_model_extra_info'

In [62]:
with mlflow.start_run(experiment_id=experiment_id,run_name=RUN_NAME) as run:
    run_id = run.info.run_id
    model_info = mlflow.sklearn.log_model(sk_model=model,artifact_path='models',registered_model_name=REGISTRY_MODEL_NAME,pip_requirements=pip_requirements,signature=signature)

MlflowException: API request to http://127.0.0.1:5001/api/2.0/mlflow/runs/create failed with exception HTTPConnectionPool(host='127.0.0.1', port=5001): Max retries exceeded with url: /api/2.0/mlflow/runs/create (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f46b19a8c70>: Failed to establish a new connection: [Errno 111] Connection refused'))